In [ ]:
from PIL import Image , ImageEnhance , ImageFilter
import google.generativeai as genai
from pdf2image import convert_from_path
import numpy as np
import os
from pdf2image import convert_from_path
from ultralytics import YOLO 
import pytesseract
import cv2

# load model
yolo_model = YOLO('')
# set model parameters
yolo_model.overrides['conf'] = 0.25  # NMS confidence threshold
yolo_model.overrides['iou'] = 0.45  # NMS IoU threshold
yolo_model.overrides['agnostic_nms'] = False  # NMS class-agnostic
yolo_model.overrides['max_det'] = 10  # maximum number of detections per image

genai.configure(api_key="")

json_model = genai.GenerativeModel("gemini-2.0-flash")


poppler_path = ""
folder_path = ""
output_path = ""
pytesseract.pytesseract.tesseract_cmd = ""
os.environ['TESSDATA_PREFIX'] =  ""
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    base_name = os.path.splitext(filename)[0]  # Get filename without extension

    # Convert each page of the PDF to an image
    pages = convert_from_path(file_path, poppler_path=poppler_path, dpi=300)

    # Loop through each page image
    for i, img in enumerate(pages):
        # Use YOLO model to detect tables in the image
        results = yolo_model(img)
        boxes = results[0].boxes.xyxy  # Get bounding boxes for detected tables

        # Create a subfolder to save cropped tables and results
        sub_folder = os.path.join(output_path, base_name)
        os.makedirs(sub_folder, exist_ok=True)

        # Loop through each detected table box
        for j, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box)  # Convert box coordinates to integers
            cropped = img.crop((x1, y1, x2, y2))  # Crop the table from the page

            # Define path to save cropped image
            cropped_path = os.path.join(sub_folder, f"page_{i+1}_table_{j+1}.jpg")

            # Extract text from the original full page image using Tesseract OCR
            extracted_text = pytesseract.image_to_string(img, config='--oem 3 --psm 4')

            # Define path to save the JSON output
            json_path = os.path.join(sub_folder, f"page_{i+1}_table_{j+1}.json")

            # Create prompt to send to the LLM for JSON conversion
            prompt = f"""
            This is a table extracted from a financial PDF:

            {extracted_text}

            Your task:
            1. ONLY if the text contains **financial data for standalone or consolidated statements**, extract and convert them into **quarter-wise JSON**.
            2. Match each quarter and its associated metrics carefully.
            3. Do not hallucinate values. Skip if the data is ambiguous or missing.

            Your output format:
            {{
            "Standalone_financial_results_for_all_months": {{
            "Quarter ended [date]": {{
            "Revenue from operations": float,
            "Other income": float,
            ...
            }},
            ...
            }},
            "Consolidated_financial_results_for_all_months": {{
            ...
            }}
            }}
            """

            # Send prompt to the LLM (json_model) for generation
            response = json_model.generate_content(prompt)

            # Skip saving if model returns a generic response
            if "The provided text" in response.text:
                continue
            else:
                # Save the LLM-generated JSON response to file
                with open(json_path, "w", encoding="utf-8") as f:
                    f.write(response.text)

c:\Users\User\Desktop\Reena\Financial_Data_Table_Extraction\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



0: 640x480 1 bordered, 1 borderless, 100.0ms
Speed: 1.8ms preprocess, 100.0ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

0: 480x640 1 borderless, 100.5ms
Speed: 2.0ms preprocess, 100.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 bordered, 1 borderless, 96.8ms
Speed: 1.5ms preprocess, 96.8ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 borderlesss, 95.1ms
Speed: 2.2ms preprocess, 95.1ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 640x480 (no detections), 95.4ms
Speed: 2.0ms preprocess, 95.4ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 90.0ms
Speed: 1.5ms preprocess, 90.0ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 85.6ms
Speed: 1.4ms preprocess, 85.6ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 (no detections), 101.0ms
Speed: 1.5ms pr